# DFM - Tranformação das séries

> Tranformando as séries cruas em séries estacionárias com as primeiras e segundas diferenças. Ainda, os dados são agregados pela média para trimestres, de modo a deixar todas as variáveis mensais em trimestrais. 

> OBS: Outras transformações dos dados de mensal para trimestral podem ser testadas.

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Lendo o arquivo dos dados

In [2]:
# Lendo o arquivo csv

now_data = pd.read_csv('../../data/now_data_raw_full.csv')
now_data.drop('Unnamed: 0', axis = 1, inplace = True)
now_data['Date'] = pd.to_datetime(now_data['Date'], format = '%Y-%m-%d')
now_data.set_index('Date', inplace = True)
now_data.rename(columns={'retail_sales':'com_sales'}, inplace=True)
now_data.shape

(249, 195)

In [3]:
# Lendo arquivo do dicionário de dados

meta_data = pd.read_csv('../../data/meta-data-full.csv')
transf = meta_data[['name_df', 'transf']]
transf.set_index('name_df', inplace = True)
transf.head()

,transf
name_df,
gdp,pctchg_12
ibc_br,diff_12
mon_gdp,diff_12
ind_prod,diff_12
ind_prod_ext,diff_12


# Tranformação dos dados

In [4]:
# Separando em grupos de tranformação 

non_diffs = list(transf[transf['transf'] == 'none'].index)
first_diff = list(transf[transf['transf'] == 'diff_12'].index)
second_diff = list(transf[transf['transf'] == 'diff_diff'].index)
second_diff12 = list(transf[transf['transf'] == 'diff_diff_12'].index)
pct_change = list(transf[transf['transf'] == 'pctchg'].index)
pct_change12 = list(transf[transf['transf'] == 'pctchg_12'].index)

# Loop para aplicar tratamentos devidos

now_data_tf = now_data.copy()

variables = now_data_tf.columns

for i in variables:
   if (i in non_diffs):
       now_data_tf[i] = now_data_tf[i]
   elif (i in first_diff):
       now_data_tf[i] = now_data_tf[i].dropna().diff(12)
   elif (i in second_diff):
        now_data_tf[i] = now_data_tf[i].dropna().diff().diff()
   elif (i in second_diff12):
        now_data_tf[i] = now_data_tf[i].dropna().diff(12).diff()
   elif (i in pct_change):
        now_data_tf[i] = now_data_tf[i].dropna().pct_change()
   elif (i in pct_change12):
        now_data_tf[i] = now_data_tf[i].dropna().pct_change(4)

now_data_tf.head(15)

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,ind_prod_nondur_goods,ind_prod_cc_materials,ind_real_sales,vehi_prod,pass_cars_prod,truck_prod,bus_prod,steel_prod,road_flow_gross_vehi,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,igpm,igpdi,incc_m,incc_di_mo,incc_di_servmat,ipatac_m,ipc_br,ipa,ipc_fipe,commdt_index_br,commdt_index_agro,commdt_index_metal,sinapi,ccb_sp,ccb_bh,ccb_rj,ccb_bsb,ccb_sal,wage_mass_ind,new_caged,caged_agro,caged_ind_ext,caged_ind_transf,caged_const,caged_com,caged_servi,hrs_wkd_ind_transf,unem_rate_pnad,total_sales,const_sales,com_sales,forn_app_sales,fuel_sales,nondur_congoods_sales,vest_sales,vehi_sales,medcare_sales,books_sales,offcomm_sales,others_sales,vehi_total_sales_fenabrave,pass_vehi_sales,light_comm_vehi_sales,truck_sales,bus_sales,vehi_total_sales_anfavea,vehi_domestic_sales,energy_comm,energy_res,energy_ind,energy_total,prod_crude_oil,prod_lgn,prod_oil_deriv,co_gas,co_glp,co_oilcomb,co_diesel,co_oilderiv_total,co_hid_alc,co_alc_total,curr_acc,idp_gdp_twelve_mon,ide_mon,net_publi_debt_states,pr_central_gov,net_debt_percent_gdp,net_debt_per_gdp_states,net_debt_per_gdp_pe,net_debt_per_gdp_cons_ps,nm_debt_cost,nm_debt_cost_int,mon_base,inter_reserv,inter_reserv_liq,mi_wkdavg,mii_balance_endp,selic_mon,interest_longterm,cdi_mon,credit_tot,credit_enter,bndes_desem_total,bndes_desem_ind_transf,bndes_desem_comserv,bndes_desem_ind_ext,exr_usd_mon_mean,household_debt,serv_conf_index_fgv,serv_curr_situ_index_fgv,serv_expec_index_fgv,ind_conf_cons_feco,ind_eco_cond_feco,ind_fut_expec_feco,emp_conf_index_fgv,emp_curr_situ_index_fgv,emp_expec_index_fgv,ind_conf_index_fgv,ind_curr_situ_index_fgv,ind_expec_index_fgv,nuci_gv,cons_conf_index_fgv,cons_curr_situ_index_fgv,cons_expec_index_fgv,expec_gdp,expec_ipca,expec_selic,expec_exr_usd,exp_fob,exp_kgood_fob,exp_comgood_fob,imp_fob,imp_kgood_fob,imp_comgood_fob,revenue_rf,brent_oil_price,ibov,nasdaq,snp_500,dji,gt_arts,gt_books,gt_finance,gt_health,gt_internet,gt_news,gt_pets,gt_science,gt_travel,gt_auto,gt_business,gt_foods,gt_lazer,gt_jobs,gt_oncomm,gt_realestate,gt_shopping,gt_bfit,gt_pceletro,gt_games,gt_home,gt_law,gt_povo,gt_ref,gt_sports,gt_crise,gt_desemprego,gt_segdesemp,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia,gt_cresc_pib,gt_inflacao,gt_juros,gt_divida,gt_falencia,gt_emprestimo,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Teste final de estacionariedade

In [5]:
# Função para realizar o teste ADF 1

def adfuller_test1(series, signif = 0.05):

# Perform Augmented Dickey-Fuller to test for Stationarity of the given series
# and print report. Null Hypothesis [H0]: Data has unit root and is non-stationary.

# series: time series in pd.Series format
# signif: significance level for P-value to reject Null Hypothesis

    x = adfuller(series, 
                 regression = 'n',            # sem constante e sem termo de tendência determinística
                 autolag = 'BIC')              # inserindo o critério bayesiano de informação para determinaro 
                                               # número de lags em cada regressão do teste
                                               # Seguimos com o critério BIC para a inclusão automática de lags como feito em Issler (2016).

    #using dictionary saves different data types (float, int, boolean)
    output = {'test_statistic': x[0], 
              'p_value': x[1], 
              'n_lags': x[2], 
              'n_observations': x[3],
              f'reject_005': x[1] < signif }

    for key, val in x[4].items():
         output[f'Critical value {key}'] = val

    return pd.Series(output)

In [6]:
# Função para realizar o teste ADF 2

def adfuller_test2(series, signif = 0.05):

# Perform Augmented Dickey-Fuller to test for Stationarity of the given series
# and print report. Null Hypothesis [H0]: Data has unit root and is non-stationary.

# series: time series in pd.Series format
# signif: significance level for P-value to reject Null Hypothesis

    x = adfuller(series, 
                 regression = 'c',            # inserindo apenas a constante
                 autolag = 'BIC')              # inserindo o critério bayesiano de informação para determinaro 
                                               # número de lags em cada regressão do teste
                                               # Seguimos com o critério BIC para a inclusão automática de lags como feito em Issler (2016).

    #using dictionary saves different data types (float, int, boolean)
    output = {'test_statistic': x[0], 
              'p_value': x[1], 
              'n_lags': x[2], 
              'n_observations': x[3],
              f'reject_005': x[1] < signif }

    for key, val in x[4].items():
         output[f'Critical value {key}'] = val

    return pd.Series(output)

In [7]:
# Função para realizar o teste ADF 3 

def adfuller_test3(series, signif = 0.05):

# Perform Augmented Dickey-Fuller to test for Stationarity of the given series
# and print report. Null Hypothesis [H0]: Data has unit root and is non-stationary.

# series: time series in pd.Series format
# signif: significance level for P-value to reject Null Hypothesis

    x = adfuller(series, 
                 regression = 'ct',            # inserindo uma constante e um termo de tendência na regressão
                 autolag = 'BIC')              # inserindo o critério bayesiano de informação para determinaro 
                                               # número de lags em cada regressão do teste
                                               # Seguimos com o critério BIC para a inclusão automática de lags como feito em Issler (2016).

    #using dictionary saves different data types (float, int, boolean)
    output = {'test_statistic': x[0], 
              'p_value': x[1], 
              'n_lags': x[2], 
              'n_observations': x[3],
              f'reject_005': x[1] < signif }

    for key, val in x[4].items():
         output[f'Critical value {key}'] = val

    return pd.Series(output)

In [8]:
# Realizando o teste ADF 1

adf_tf = pd.DataFrame()
cols = now_data_tf.columns

for column in cols:
  adf_test = adfuller_test1(now_data_tf[column].dropna())
  adf_tf[column] = adf_test

adf_tf_t = adf_tf.T
adf_tf_t[adf_tf_t['reject_005'] == False]

C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\157033840.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\157033840.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\157033840.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

,test_statistic,p_value,n_lags,n_observations,reject_005,Critical value 1%,Critical value 5%,Critical value 10%
mon_gdp,1.045358,0.92205,12,224,False,-2.575794,-1.942263,-1.615686
total_sales,-1.924317,0.051849,12,224,False,-2.575794,-1.942263,-1.615686
energy_res,-1.713517,0.082086,12,224,False,-2.575794,-1.942263,-1.615686
revenue_rf,-1.21805,0.20466,12,222,False,-2.575885,-1.942275,-1.615676


In [9]:
# Realizando o teste ADF 2

adf_tf = pd.DataFrame()
cols = now_data_tf.columns

for column in cols:
  adf_test = adfuller_test2(now_data_tf[column].dropna())
  adf_tf[column] = adf_test

adf_tf_t = adf_tf.T
adf_tf_t[adf_tf_t['reject_005'] == False]

C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\1406786598.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\1406786598.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\1406786598.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using

,test_statistic,p_value,n_lags,n_observations,reject_005,Critical value 1%,Critical value 5%,Critical value 10%
exr_usd_mon_mean,-2.435846,0.131874,12,224,False,-3.459885,-2.874531,-2.573694
gt_inflacao,-2.559255,0.101714,12,200,False,-3.463476,-2.876102,-2.574532


In [10]:
# Realizando o teste ADF 3

adf_tf = pd.DataFrame()
cols = now_data_tf.columns

for column in cols:
  adf_test = adfuller_test3(now_data_tf[column].dropna())
  adf_tf[column] = adf_test

adf_tf_t = adf_tf.T
adf_tf_t[adf_tf_t['reject_005'] == False]

C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\115083059.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\115083059.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adf_tf[column] = adf_test
C:\Users\Henri\AppData\Local\Temp\ipykernel_18692\115083059.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

,test_statistic,p_value,n_lags,n_observations,reject_005,Critical value 1%,Critical value 5%,Critical value 10%


> Nenhuma variável apareceu no conjunto de séries que falhou em rejeitar a hipótese nula de que a série é não estacionária e que possui raíz unitária. Logo, os procedimentos para estacionarizar foram bem sucedidos.

# Agregando as séries temporais

In [11]:
'''# Agregando as séries pela média e substituindo NAs pela mediana

qdata_tf = (now_data_tf.groupby(pd.PeriodIndex(now_data_tf.index, freq = 'Q'), axis = 0)).mean()
cols = qdata_tf.columns

for col in cols:
    qdata_tf[col].fillna(qdata_tf[col].median(), inplace = True)

qdata_tf.head(10)
qdata_tf.shape'''

"# Agregando as séries pela média e substituindo NAs pela mediana\n\nqdata_tf = (now_data_tf.groupby(pd.PeriodIndex(now_data_tf.index, freq = 'Q'), axis = 0)).mean()\ncols = qdata_tf.columns\n\nfor col in cols:\n    qdata_tf[col].fillna(qdata_tf[col].median(), inplace = True)\n\nqdata_tf.head(10)\nqdata_tf.shape"

# Salvando a base de dados nova tranformada: mensal e apenas PIB trimestral 

In [12]:
data_tf = now_data_tf.copy()
cols = data_tf.columns[1:]

for col in cols:
    data_tf[col].fillna(data_tf[col].median(), inplace = True)

data_tf

,gdp,ibc_br,mon_gdp,ind_prod,ind_prod_ext,ind_prod_transf,ind_cap_goods,ind_prod_int_goods,ind_prod_man,ind_prod_dur_goods,ind_prod_nondur_goods,ind_prod_cc_materials,ind_real_sales,vehi_prod,pass_cars_prod,truck_prod,bus_prod,steel_prod,road_flow_gross_vehi,ipca,ipca_ex,ipca_exi,ipca_exii,ipca_exiii,inpc,igpm,igpdi,incc_m,incc_di_mo,incc_di_servmat,ipatac_m,ipc_br,ipa,ipc_fipe,commdt_index_br,commdt_index_agro,commdt_index_metal,sinapi,ccb_sp,ccb_bh,ccb_rj,ccb_bsb,ccb_sal,wage_mass_ind,new_caged,caged_agro,caged_ind_ext,caged_ind_transf,caged_const,caged_com,caged_servi,hrs_wkd_ind_transf,unem_rate_pnad,total_sales,const_sales,com_sales,forn_app_sales,fuel_sales,nondur_congoods_sales,vest_sales,vehi_sales,medcare_sales,books_sales,offcomm_sales,others_sales,vehi_total_sales_fenabrave,pass_vehi_sales,light_comm_vehi_sales,truck_sales,bus_sales,vehi_total_sales_anfavea,vehi_domestic_sales,energy_comm,energy_res,energy_ind,energy_total,prod_crude_oil,prod_lgn,prod_oil_deriv,co_gas,co_glp,co_oilcomb,co_diesel,co_oilderiv_total,co_hid_alc,co_alc_total,curr_acc,idp_gdp_twelve_mon,ide_mon,net_publi_debt_states,pr_central_gov,net_debt_percent_gdp,net_debt_per_gdp_states,net_debt_per_gdp_pe,net_debt_per_gdp_cons_ps,nm_debt_cost,nm_debt_cost_int,mon_base,inter_reserv,inter_reserv_liq,mi_wkdavg,mii_balance_endp,selic_mon,interest_longterm,cdi_mon,credit_tot,credit_enter,bndes_desem_total,bndes_desem_ind_transf,bndes_desem_comserv,bndes_desem_ind_ext,exr_usd_mon_mean,household_debt,serv_conf_index_fgv,serv_curr_situ_index_fgv,serv_expec_index_fgv,ind_conf_cons_feco,ind_eco_cond_feco,ind_fut_expec_feco,emp_conf_index_fgv,emp_curr_situ_index_fgv,emp_expec_index_fgv,ind_conf_index_fgv,ind_curr_situ_index_fgv,ind_expec_index_fgv,nuci_gv,cons_conf_index_fgv,cons_curr_situ_index_fgv,cons_expec_index_fgv,expec_gdp,expec_ipca,expec_selic,expec_exr_usd,exp_fob,exp_kgood_fob,exp_comgood_fob,imp_fob,imp_kgood_fob,imp_comgood_fob,revenue_rf,brent_oil_price,ibov,nasdaq,snp_500,dji,gt_arts,gt_books,gt_finance,gt_health,gt_internet,gt_news,gt_pets,gt_science,gt_travel,gt_auto,gt_business,gt_foods,gt_lazer,gt_jobs,gt_oncomm,gt_realestate,gt_shopping,gt_bfit,gt_pceletro,gt_games,gt_home,gt_law,gt_povo,gt_ref,gt_sports,gt_crise,gt_desemprego,gt_segdesemp,gt_empregos,gt_trabalho,gt_fgts,gt_renda,gt_salario,gt_economia,gt_cresc_pib,gt_inflacao,gt_juros,gt_divida,gt_falencia,gt_emprestimo,gt_politica,gt_aluguel,gt_seguro,gt_governo,gt_viagem,gt_empreendedor,gt_empresa,gt_credito,gt_dolar,gt_financiamento
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2002-01-01,NaN,2.45,29367.7,0.7,2.8,0.8,3.8,0.3,0.8,2.3,0.5,-3.9,0.7,8667.0,7376.0,1015.0,52.0,3.47,-4.000000e-02,0.03,-0.01,0.00,0.01,0.00,0.02,-0.06,0.03,-0.02,0.00,-0.02,-0.05,0.03,0.03,-0.01,0.12,0.235,14.58,-0.04,-0.01,-0.02,0.375,1.10,-0.25,0.0,1355.0,20926.0,-71.0,1625.0,3050.5,27.5,2340.0,0.0,0.000000e+00,3.33,3.40,3.66,6.28,0.11,2.51,1.24,3.16,-0.055,0.31,5.55,4.67,6619.0,3993.0,1624.0,429.0,37.0,6896.0,3830.0,231.0,330.0,195.0,976.0,61.0,3.0,67.0,11.0,1.0,-5.0,-4.0,-8.0,15.0,19.0,-78.0,1.000000e-02,48.5,283.42,440.65,-0.24,0.005,0.000000e+00,-1.36,-449.0,-0.09,13935437.0,15.5,-163.0,18315348.0,629973.5,-5.551115e-17,0.00,-0.005,-2627.5,-284.5,468.0,-155.0,644.0,7.0,0.0202,-3.552714e-15,-2.000000e-01,-0.2,-0.6,-0.01,-0.63,0.15,0.5,0.7,-0.6,0.3,1.3,-1.1,0.3,7.105427e-15,3.000000e-01,-3.500000e-01,0.000000e+00,0.1300,-0.25,0.0500,74.691883,78.022109,193.828495,1000.460209,120.192831,-0.359005,6394.011055,0.013375,0.006448,0.015446,0.011576,0.009538,-0.250,0.0625,1.750,0.125,-0.1875,1.250,0.125,-0.375,2.375,2.125,2.000,2.875,2.125,0.000,0.125,2.250,0.000,0.125,-0.3125,-0.0625,-0.0625,1.000,0.0625,2.125,2.000,0.625,-0.500,0.750,-3.375,0.000,0.000,-0.125,3.625,-1.625,-0.625,-0.250,1.500,2.375,-1.375,0.125,0.375,0.750,0.375,0.000,1.375,-0.250,-1.125,0.000,2.3

In [13]:
data_tf.shape

(249, 195)

In [14]:
'''data_tf2 = data_tf.copy()

# Criando lags do PIB

data_tf2['gdp_1'] = data_tf2['gdp'].shift(1)
data_tf2['gdp_2'] = data_tf2['gdp'].shift(2)
data_tf2['gdp_3'] = data_tf2['gdp'].shift(3)
data_tf2['gdp_4'] = data_tf2['gdp'].shift(4)
data_tf2['gdp_5'] = data_tf2['gdp'].shift(5)
data_tf2['gdp_6'] = data_tf2['gdp'].shift(6)
data_tf2['gdp_7'] = data_tf2['gdp'].shift(7)
data_tf2['gdp_8'] = data_tf2['gdp'].shift(8)
data_tf2'''

"data_tf2 = data_tf.copy()\n\n# Criando lags do PIB\n\ndata_tf2['gdp_1'] = data_tf2['gdp'].shift(1)\ndata_tf2['gdp_2'] = data_tf2['gdp'].shift(2)\ndata_tf2['gdp_3'] = data_tf2['gdp'].shift(3)\ndata_tf2['gdp_4'] = data_tf2['gdp'].shift(4)\ndata_tf2['gdp_5'] = data_tf2['gdp'].shift(5)\ndata_tf2['gdp_6'] = data_tf2['gdp'].shift(6)\ndata_tf2['gdp_7'] = data_tf2['gdp'].shift(7)\ndata_tf2['gdp_8'] = data_tf2['gdp'].shift(8)\ndata_tf2"

### Salvando em .csv os dados transformados

In [15]:
# now_data_tf.to_csv('now_data_tf.csv')
# now_data_tf.to_excel('now_data_tf.xlsx')

# qdata_tf.to_csv('../../data/qdata_tf.csv')
# qdata_tf.to_excel('../../data/qdata_tf.xlsx')

data_tf.to_csv('../../data/data_full_tf12.csv')
# data_tf.to_excel('../../data/data_tf.xlsx')